In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
import nltk.data
import proj_base
import scipy.spatial.distance as scpd
from numpy.linalg import inv
import nltk.sentiment
import time
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from textblob import TextBlob
import pprint as pp
import Sentiment_analysis

import sklearn.feature_extraction

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data = proj_base.getStandardData(numFiles=25)

unchanged = data.copy()
aspect = "Rooms"
proj_base.aspect = aspect
data.shape

(2447, 13)

In [3]:
data = data.apply(Sentiment_analysis.addSentimentScores, axis = 1)

In [4]:
def isFeatureVecNull(fv):
    if sum(pd.notnull(fv)) == len(fv):
        return True
    return False


def getFeatureVec(review, useAspectSentences=False):
    if useAspectSentences:
        rev2words = sentences2wordlist(review)
    else:
        rev2words = review2wordlist(review, tokenizer)
    return makeFeatureVec(rev2words, model, num_features)


def doPSPCosine(r1, r2):
    return  1 - scpd.cosine([r1, 1-r1], [r2, 1-r2])
    #return scpd.cosine([r1, 1-r1], [r2, 1-r2])

def getAllPSP(rev, allRevs):
    return allRevs.apply(doPSPCosine, args=(rev,))

def getAllSims(rev, allRevs):
    return allRevs.apply(cosSim, args=(rev,))

def cosSim(r1, r2):
    return 1-scpd.cosine(r1, r2)

def buildSimilarityMatrix(data):
    numNodes = data.shape[0]
    sims = data["featureVec"].apply(getAllSims, args=(data["featureVec"],))
    
                
    return sims


def getPSP(rev, useAspectSentences=False):
    sentences = tokenizer.tokenize(rev.strip())
    count = 0
    ps = 0
    for s in sentences:
        count += 1
        sentiment = TextBlob(s)
        #print(sentiment)
        #print(sentiment.polarity)
        ps += sentiment.polarity
        
    return ps


def buildPSPSimilarityMatrix(data):
    numNodes = data.shape[0]
    sims = data["psp"].apply(getAllPSP, args=(data["psp"],))
    
  
                
    return sims

time1 = time.time()

#similarities = buildSimilarityMatrix(data)
#g = buildGraph(data, similarities)
time2 = time.time()

print("time taken", time2-time1)
#similarities

def getDataHist(data):
    nums = [0]*np.amax(data[aspect])
    for i in range(len(data)):
        score = data.loc[i][aspect]
        if not np.isnan(score):
            nums[int(score)-1] += 1
    return nums


def getBalancedSubsample(data, numUnlabeled=100):
    nums = getDataHist(data)
    #print(nums)
    toGrab = np.amin(nums)
    
    first = data[data[aspect] == 1.0][0:toGrab]
    second = data[data[aspect] == 2.0][0:toGrab]
    third = data[data[aspect] == 3.0][0:toGrab]
    fourth = data[data[aspect] == 4.0][0:toGrab]
    fifth = data[data[aspect] == 5.0][0:toGrab]
    unlabeled = data[pd.isnull(data[aspect])][0:numUnlabeled]
   
    df = pd.concat([first, second, third, fourth, fifth, unlabeled], ignore_index=True)
    df = df.reindex(np.random.permutation(df.index))
    df.reset_index(drop=True, inplace=True)
    return df




time taken 4.291534423828125e-05


In [5]:

def calcMinLossPredictions(data):
    #define constants
    #sim = buildSimilarityMatrix(data)
    sim = buildPSPSimilarityMatrix(data)
    sim = (sim - sim.mean()) / (sim.max() - sim.min())
    n = data.shape[0]
    print("Num data", n)
    a = 5
    b = 1
    
    k = int(n/5)
    k_p = 1
    M = 300
    beta = a*1.0/b
    alpha = a*k + b*k_p
    isLabeled = np.isnan(data[aspect]) == False 
    #k = int(.2*sum(isLabeled))
    
    print("num labeled", sum(isLabeled))
    #print(isLabeled)
    y = data["y"].values
    isLabeled = isLabeled & np.invert(np.isnan(y))
    print("num labeled", sum(isLabeled))
    y = np.nan_to_num(y)
    np.place(y, y==0, 3)
    #print(y)
    kNN = np.zeros((n, n), float)
    k_pNN = np.zeros((n, n), float)
    #build kNN matrix
    for i in range(n):
        if not isLabeled[i]:
            #LABELED NEIGHBORS
            candidates = np.multiply(isLabeled,sim[i])
            kthClosest = sorted(candidates, reverse=True)[k-1]
            for j in range(n):
                thisSimilar = candidates[j]
                if thisSimilar >= kthClosest and i != j:
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect],data.loc[i]["psp"], data.loc[j]["psp"])
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect])
                    kNN[i][j] = 1

            candidates = np.multiply(np.invert(isLabeled),sim[i])
            kthClosest = sorted(candidates, reverse=True)[k_p]
            for j in range(n):
                thisSimilar = candidates[j]
                if thisSimilar >= kthClosest and i != j:
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect],data.loc[i]["psp"], data.loc[j]["psp"])
                    #print("similar, notLabled",i,j, data.loc[i][aspect], data.loc[j][aspect])
                    k_pNN[i][j] = 1
    #pp.pprint( kNN)

    
    c_diag = np.ones(len(data)) + M*isLabeled
    C = np.zeros((n, n), int)
    
    np.fill_diagonal(C, c_diag)
    #print(C)
    W_p = np.zeros((n,n), float)
    for i in range(n):
        for j in range(n):
            if isLabeled[i]:
                W_p[i][j] = 0
            elif isLabeled[j] and kNN[i][j] == 1:
                W_p[i][j] = sim[i][j]
            elif not isLabeled[j] and k_pNN[i][j] == 1:
                W_p[i][j] = beta*sim[i][j]
    #print("W_p",W_p)        
    W = np.maximum(W_p, np.transpose(W_p))
    #print("W ", W)
    D = np.zeros((n, n), float)
    d_diag = np.zeros(n)
    for i in range(n):
        d_diag[i] =  sum(W[i])
    np.fill_diagonal(D, d_diag)
    #pp.pprint(D)
    delta = D - W
    
    constant = alpha*1.0/(k + k_p*beta)
    toInv = C + constant*delta
    inverse = inv(toInv)
    
    C_y = C.dot(y)
    preds = inverse.dot(C_y)
    
    #print(preds)
    #return np.ceil(preds)
    return np.round(preds)


In [6]:

data["sentences"] = data["Content"].apply(tokenizer.tokenize)

#sim

In [51]:
data = proj_base.getStandardData(numFiles=50)
getDataHist(data)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[63, 117, 310, 670, 1090]

In [52]:
aspect = "Location"
#data = unchanged.copy()
proj_base.aspect = aspect
scoreCols = [col for col in data.columns if  "score" in col]
X_senti = data[scoreCols]

In [53]:
trainLoc = pd.notnull(data[aspect])
testLoc = np.invert(trainLoc)
print(len(trainLoc), len(testLoc), data.shape[0])

TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,2),
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(data['Content'])
X_tfidf = X_tfidf.toarray()
newX = np.concatenate((X_senti,X_tfidf), axis = 1)


3900 3900 3900


In [54]:
trainTake = [i for i in range(len(trainLoc)) if trainLoc[i]]
testTake = [i for i in range(len(trainLoc)) if not trainLoc[i]]

lin_clf = sklearn.svm.LinearSVC()
X_train = newX[trainTake] 
X_test = newX[testTake]
y_train = data[aspect][trainLoc] 
y_test = data[aspect][testLoc]
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(X_test)
print(len(y_predicted))

1650


In [ ]:
data = unchanged.copy()

In [ ]:
print(sum(pd.isnull(data[aspect]), sum(pd.notnull(data[aspect]))))

In [55]:
getDataHist(data)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[63, 117, 310, 670, 1090]

In [56]:
unLabIndex = 0

def addY(rev):
    global unLabIndex
    if np.isnan(rev[aspect]):
        rev["y"] = y_predicted[unLabIndex]
        unLabIndex += 1
    else:
        rev["y"] = rev[aspect]
    return rev
        
data = data.apply(addY, axis=1)

In [57]:

proj_base.aspect = aspect
data = getBalancedSubsample(data, numUnlabeled=75)

getDataHist(data)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:68: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[63, 63, 63, 63, 63]

In [ ]:
data.shape

In [26]:
proj_base.aspectSegmentationChiSquared(data)
proj_base.seeds

{'Business service': ['business',
  'center',
  'computer',
  'internet',
  'access',
  'wireless',
  'convention',
  'free',
  'fitness',
  'computers',
  'westlake',
  'leisure',
  'wired'],
 'Check in / front desk': ['stuff',
  'check',
  'help',
  'reservation',
  'told',
  'early',
  'upon',
  'arrival',
  'desk',
  'asked',
  'front',
  'clerk',
  'staff'],
 'Cleanliness': ['clean',
  'dirty',
  'maintain',
  'smell',
  'comfortable',
  'room',
  'garbage',
  'view',
  'bed',
  'floor',
  'spacious',
  'king',
  'bathroom'],
 'Location': ['location',
  'traffic',
  'minute',
  'restaurant',
  'great',
  'good',
  'pike',
  'market',
  'place',
  'restaurants',
  'distance',
  'downtown',
  'walking'],
 'Rooms': ['room',
  'suite',
  'view',
  'bed',
  'floor',
  'comfortable',
  'clean',
  'spacious',
  'king',
  'given',
  'beds',
  'bathroom',
  'space'],
 'Service': ['service',
  'food',
  'breakfast',
  'buffet',
  'continental',
  'customer',
  'restaurant',
  'good',
  'ate

In [58]:


def filterReviewSentencesByWords(rev, words):
    #tokenize review into sentences
    sentences = tokenizer.tokenize(rev["Content"])
    
    #print(sentences)
    aspSentences = []
    for s in sentences:
        wordlist = re.sub("[^a-zA-Z]"," ", s).split()
        intersect = set(wordlist).intersection(set(words))
        #print(s)
        if len(intersect) != 0:
            
            aspSentences.append(s)

    if len(aspSentences) == 0:
        rev["aspectSentences"] = rev["Content"]
    else:
        rev["aspectSentences"] = ' .'.join(aspSentences)

    return rev

data = data.apply(filterReviewSentencesByWords, axis=1, args=(proj_base.seeds[aspect],))

data["psp"] = data["aspectSentences"].apply(getPSP)
data.head()

,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall,y,aspectSentences,psp
0,1.0,"Mar 5, 2008",3.0,3.0,2.0,malisam71,3.0,NaN,2.0,tiny just a warning before you go. the parking...,2.0,NaN,3.0,3.0,tiny just a warning before you go. the parking...,0.071042
1,5.0,"Aug 31, 2008",5.0,5.0,5.0,Musiclover9,5.0,1.0,5.0,"high quality, low cost at the university inn i...",5.0,1.0,5.0,5.0,the rooms are set up with great attention to d...,0.978333
2,1.0,"Jul 6, 2008",1.0,1.0,1.0,christiem,1.0,0.0,3.0,"sadly, all of these bad reviews are correct. w...",1.0,1.0,1.0,1.0,our friends also got gift certificates for the...,0.550000
3,4.0,"Nov 5, 2008",3.0,5.0,NaN,7Marini8,4.0,NaN,5.0,great place to stay we stayed two nights at th...,4.0,NaN,4.0,3.0,great place to stay we stayed two nights at th...,1.620833
4,5.0,"Oct 11, 2006",5.0,5.0,2.0,Jacktraveler,5.0,5.0,5.0,excellent choice great experience at this litt...,4.0,5.0,5.0,5.0,excellent choice great experience at this litt...,0.934643


In [28]:
def doCrossValidation(data, nfolds):
    unchanged = data.copy()
    total = data.shape[0]
    num_test = int(total/nfolds)

    rows_list = []
    for f in range(nfolds):
        #reset data
        
        data[aspect] = unchanged[aspect]
        test_ix = list(range(f*num_test, (f+1)*num_test))
        train_ix = [t for t in range(0, total) if t not in test_ix]
        
        print("cross fold nr ", f, "testSize", len(test_ix))
        #split data into train and test
        test = data.loc[test_ix]
        train = data.loc[train_ix]
        
        
        truth = unchanged[aspect][test_ix]
  
        
        for t in test_ix:
            data.set_value(t, "y", np.nan)
  

        fullpreds = calcMinLossPredictions(data)
        predicted = fullpreds[test_ix]
        print("DONE with fold", f+1, np.mean(predicted == truth))
 
        
        for p in test_ix:
            #print("pred v truth:", p,fullpreds[p], data.loc[p]["y"], data.loc[p][aspect])
            if not np.isnan(data.loc[p][aspect]):
                #print("huh")
                rows_list.append({"truth": data.loc[p][aspect], "predicted": fullpreds[p]})
            
        
        for t in test_ix:
            data.set_value(t, "y", unchanged.loc[t]["y"])
            #print("SET BACK", data.loc[t]["y"], unchanged.loc[t]["y"])
    return pd.DataFrame(rows_list) 

        

In [59]:
preds = doCrossValidation(data, 5)
print(np.mean(preds["predicted"] == preds["truth"]))


cross fold nr  0 testSize 78
Num data 390
num labeled 315
num labeled 252
DONE with fold 1 0.230769230769
cross fold nr  1 testSize 78
Num data 390
num labeled 315
num labeled 252
DONE with fold 2 0.24358974359
cross fold nr  2 testSize 78
Num data 390
num labeled 315
num labeled 255
DONE with fold 3 0.205128205128
cross fold nr  3 testSize 78
Num data 390
num labeled 315
num labeled 249
DONE with fold 4 0.205128205128
cross fold nr  4 testSize 78
Num data 390
num labeled 315
num labeled 252
DONE with fold 5 0.192307692308
0.266666666667


In [60]:

middle = preds[preds["truth"] > 1]
middle = preds[preds["truth"] < 5]
print(np.mean(preds["predicted"] == preds["truth"]))
print(np.mean(middle["predicted"] == middle["truth"]))
preds.head(15)

0.266666666667
0.333333333333


,predicted,truth
0,2.0,3.0
1,3.0,5.0
2,3.0,1.0
3,4.0,3.0
4,3.0,5.0
5,3.0,1.0
6,3.0,4.0
7,3.0,1.0
8,2.0,1.0
9,3.0,3.0


In [ ]:
print(sum(preds["predicted"] == 1), sum(preds["predicted"] == 5))

In [31]:
preds.to_csv(aspect+"Graph.csv")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,"{0:.2f}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Test confusion matrix function
y_test=preds["truth"]
y_pred=preds["predicted"]
class_names=['1','2','3','4','5']
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()